In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [1]:
#import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# import credentials
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
#connect to salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
#connect to des mysql db
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL Data

In [5]:
#reads in course table from mysql
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [6]:
#rename columns from course table
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [7]:
#Dropping ID_Course
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [8]:
#convert dataframe to a list of dictionaries
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [11]:
#delets course data
class_records = sf.query("SELECT Id FROM Course__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
recs_to_delete

sf.bulk.Course__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a004x000003UPCZAA4', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003UPCeAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003UPCjAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003UPCoAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003UPCtAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003UPCyAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003UPD3AAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003UPD8AAO', 'errors': []}]

In [55]:
 #creating records for each dic in dic list
for rec in course_data_records:
    #taking a key out of dictionary putting into new dic format
    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [14]:
#read in student data
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head(30)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F
5,38,25006014,Dartling,Precious,Mariette,None,F
6,39,25007528,Dartling,Cindy,Patty,None,F
7,40,25003778,Dartling,Carlie,Raelene,None,F
8,41,25003605,Dartling,Clementine,Kendall,None,F
9,42,25002056,Dartling,Amirah,Angela,None,F


In [18]:
#rename columns from student table
student_data_df.rename(columns={
    'StudentID':'Student_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
    'Gender': 'Gender__c',
}, inplace=True)

#Dropping ID_Student
student_data_df = student_data_df[['Student_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c', 'Gender__c']]
student_data_df.head()

,Student_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [19]:
#convert dataframe to a list of dictionaries
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'Student_ID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mariette',
  'Birth_Date__c': None,
  'Gender_

In [22]:
#delets student data
student_records = sf.query("SELECT Id FROM Student__c")
recs_to_delete = [{'Id': r['Id']} for r in student_records['records']]
recs_to_delete

sf.bulk.Student__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a024x000002gVf1AAE', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVf6AAE', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfBAAU', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfGAAU', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfLAAU', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfQAAU', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfVAAU', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfaAAE', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVffAAE', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfkAAE', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfpAAE', 'errors': []},
 {'success': True, 'created': False, 'id': 'a024x000002gVfuAAE', 'errors': []},
 {'success': True, 'created': False, 'id

In [21]:
 #creating student records for each dic in dic list
for rec in student_data_records:
    #taking a key out of dictionary putting into new dic format
    record = {
        'Student_ID__c':rec['Student_ID__c'],
        'Last_Name__c':rec['Last_Name__c'],
        'First_Name__c':rec['First_Name__c'],
        'Middle_Name__c':rec['Middle_Name__c'], 
        'Birth_Date__c':rec['Birth_Date__c'],
        'Gender__c': rec['Gender__c'],
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

In [42]:
#read in staff data
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [43]:
#rename columns from student table
staff_data_df.rename(columns={
    'EmployeeID':'Employee_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
}, inplace=True)

#Dropping ID_Staff
staff_data_df = staff_data_df[['Employee_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c']]
staff_data_df.head()

,Employee_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [44]:
#format bithdate
#staff_data_df['Birth_Date__c'] = pd.to_datetime(staff_data_df['Birth_Date__c']).dt.date

#convert dataframe to a list of dictionaries
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'Employee_ID__c': '000184220',
  'Last_Name__c': 'Luongo',
  'First_Name__c': 'Darick',
  'Middle_Name__c': 'Nico',
  'Birth_Date__c': datetime.date(1995, 7, 5)},
 {'Employee_ID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'First_Name__c': 'Gemini',
  'Middle_Name__c': 'Blair',
  'Birth_Date__c': datetime.date(1992, 4, 22)},
 {'Employee_ID__c': '000160655',
  'Last_Name__c': 'Williams',
  'First_Name__c': 'Dartanion',
  'Middle_Name__c': 'De Angelo',
  'Birth_Date__c': datetime.date(1993, 5, 21)},
 {'Employee_ID__c': '000159108',
  'Last_Name__c': 'Rodney',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Nicole',
  'Birth_Date__c': datetime.date(1994, 6, 7)}]

In [51]:
#delets student data
staff_records = sf.query("SELECT Id FROM Staff__c")
recs_to_delete = [{'Id': r['Id']} for r in staff_records['records']]
recs_to_delete

sf.bulk.Staff__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a034x000002idXxAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a034x000002idZdAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a034x000002idZiAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a034x000002idZnAAI', 'errors': []}]

In [50]:
#creating staff records for each dic in dic list
for rec in staff_data_records:
    #taking a key out of dictionary putting into new dic format
    record = {
        'Employee_ID__c':rec['Employee_ID__c'],
        'Last_Name__c':rec['Last_Name__c'],
        'First_Name__c':rec['First_Name__c'],
        'Middle_Name__c':rec['Middle_Name__c'], 
        'Birth_Date__c':str(rec['Birth_Date__c'])
    } 
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

1995-07-05
1992-04-22
1993-05-21
1994-06-07


## Look up Tables

In [56]:
#create course lookup table
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003UPIj', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a004x000003UPJ3', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003UPIt', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003UPJI', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003UPJD', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003UPIy', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003UPJ8', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003UPIo', 'Course_Code__c': 'BC-WEBDEV'}]

In [57]:
#convert to dataframe
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003UPIj,BC-DATAVIZ
1,a004x000003UPJ3,CIS-405
2,a004x000003UPIt,BC-UIUX
3,a004x000003UPJI,CIS-435
4,a004x000003UPJD,CIS-430
5,a004x000003UPIy,CIS-349
6,a004x000003UPJ8,CIS-438
7,a004x000003UPIo,BC-WEBDEV


In [58]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [59]:
#rename columns from class table
class_data_df.rename(columns={
    'ID_Course':'ID_Course__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['ID_Course__c','Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,ID_Course__c,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


In [62]:
#merge MySQL Class table to salesforce FK lookup
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left', on='Course_Code__c')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,ID_Course__c_x,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c_y
0,1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003UPIj
1,1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003UPIj
2,2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003UPIo
3,3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003UPIt


In [63]:
#convert dataframe to a list of dictionaries
class_data_records = class_data_df.to_dict('records')
class_data_records

[{'ID_Course__c_x': 1,
  'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c_y': 'a004x000003UPIj'},
 {'ID_Course__c_x': 1,
  'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'ID_Course__c_y': 'a004x000003UPIj'},
 {'ID_Course__c_x': 2,
  'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c_y': 'a004x000003UPIo'},
 {'ID_Course__c_x': 3,
  'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c_y': 'a004x000003UPIt'}]

In [65]:
#creating class records for each dic in dic list
#Insert records into salesforce
for rec in class_data_records:
    #taking a key out of dictionary putting into new dic format
    record = {
        'ID_Course__c': rec['ID_Course__c_y'],
        'Section__c':rec['Section__c'],
        'Start_Date__c':str(rec['Start_Date__c']),
        'End_Date__c':str(rec['End_Date__c'])
    } 
    try:
        sf.Class__c.create(record)
    except Exception as e:
        print(e)